In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline

In [13]:
import seaborn as sns


In [2]:
train = pd.read_csv("Train.csv")

In [3]:
test = pd.read_csv("Test.csv")

In [4]:
train.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,06-05-2011 16:54,3.75,14056,35
1,6349,1300,3682,6,11-05-2011 07:35,1.95,13098,35
2,16783,2178,1939,4,20-11-2011 13:20,5.95,15044,35
3,16971,2115,2983,1,22-11-2011 12:07,0.83,15525,35
4,6080,1210,2886,12,06-05-2011 09:00,1.65,13952,35


In [7]:
train.describe()

,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
count,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000,284780.000000
mean,9955.394083,1573.386807,2023.955573,12.028468,3.451216,15288.224278,32.823846
std,5551.519138,843.604991,1089.812078,295.111588,78.399541,1714.336529,6.743602
min,0.000000,0.000000,0.000000,-80995.000000,0.000000,12346.000000,0.000000
25%,5069.000000,939.000000,1141.000000,2.000000,1.250000,13953.000000,35.000000
50%,10310.000000,1521.000000,1987.000000,5.000000,1.950000,15152.000000,35.000000
75%,14657.000000,2106.000000,2945.000000,12.000000,3.750000,16794.000000,35.000000
max,22188.000000,3683.000000,3895.000000,80995.000000,38970.000000,18287.000000,36.000000


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    284780 non-null  int64  
 1   StockCode    284780 non-null  int64  
 2   Description  284780 non-null  int64  
 3   Quantity     284780 non-null  int64  
 4   InvoiceDate  284780 non-null  object 
 5   UnitPrice    284780 non-null  float64
 6   CustomerID   284780 non-null  int64  
 7   Country      284780 non-null  int64  
dtypes: float64(1), int64(6), object(1)
memory usage: 17.4+ MB


In [14]:
#Separating the categorical and Numerical columns

cat_cols = train.select_dtypes(include = ['object','category']).columns.tolist()
print(cat_cols)

num_cols = train.select_dtypes(include =['int64','float64']).columns.tolist()


['InvoiceDate']


In [15]:
print(num_cols)

['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'UnitPrice', 'CustomerID', 'Country']


In [17]:
train = train.drop(['InvoiceDate','InvoiceNo'],1,inplace = False)

In [18]:
train.head()

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
0,1583,144,3,3.75,14056,35
1,1300,3682,6,1.95,13098,35
2,2178,1939,4,5.95,15044,35
3,2115,2983,1,0.83,15525,35
4,1210,2886,12,1.65,13952,35


In [19]:
corr_matrix = train.corr(method='pearson')
corr_matrix

,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
StockCode,1.000000,-0.013230,-0.000942,0.020073,0.002887,0.006242
Description,-0.013230,1.000000,0.000194,-0.000424,-0.005227,-0.014791
Quantity,-0.000942,0.000194,1.000000,-0.000924,-0.002930,-0.008001
UnitPrice,0.020073,-0.000424,-0.000924,1.000000,-0.004932,-0.005378
CustomerID,0.002887,-0.005227,-0.002930,-0.004932,1.000000,0.388198
Country,0.006242,-0.014791,-0.008001,-0.005378,0.388198,1.000000


In [20]:
#correlation with unit price
corr_matrix.UnitPrice.sort_values(ascending=False)

UnitPrice      1.000000
StockCode      0.020073
Description   -0.000424
Quantity      -0.000924
CustomerID    -0.004932
Country       -0.005378
Name: UnitPrice, dtype: float64

In [21]:
#Take targate variable into y
y = train['UnitPrice']
X = train.drop('UnitPrice',axis = 1)

from sklearn.preprocessing import PowerTransformer
y = y.values.reshape(-1,1)
# power transform the raw data
power = PowerTransformer(method='yeo-johnson', standardize=True)
y = power.fit_transform(y)

In [22]:
# Split data into train and test format
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

In [23]:
# Using K_Nearest neighbour regressor
# running for different K values to know which yields the max accuracy.
from sklearn.neighbors import KNeighborsRegressor
score = []
for k in range(1,20):    
    clf = KNeighborsRegressor(n_neighbors = k,  weights = 'distance', p=1)
    clf.fit(X_train, y_train)
    score.append(clf.score(X_test, y_test))

In [24]:
k_max = score.index(max(score))+1


In [25]:
k_max

3

In [27]:
print( "At K = {}, Max Accuracy = {}".format(k_max, max(score)*100))


At K = 3, Max Accuracy = 82.6323452158664


In [28]:
clf = KNeighborsRegressor(n_neighbors = k_max,  weights = 'distance', p=1)
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test ))   
y_pred = clf.predict(X_test)

0.826323452158664


In [29]:
test.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country
0,3139,1709,1079,3,2011-02-22 15:22:00,16161.0,35
1,617,510,3457,1,2010-12-08 12:46:00,17341.0,35
2,14653,604,694,36,2011-10-25 13:53:00,15158.0,35
3,8634,1478,3473,2,2011-06-27 12:38:00,16033.0,35
4,15546,3216,871,1,2011-11-06 16:14:00,15351.0,35


In [30]:
test = test.drop(['InvoiceDate','InvoiceNo'],1, inplace= False)
test.head()

,StockCode,Description,Quantity,CustomerID,Country
0,1709,1079,3,16161.0,35
1,510,3457,1,17341.0,35
2,604,694,36,15158.0,35
3,1478,3473,2,16033.0,35
4,3216,871,1,15351.0,35


In [31]:
# apply the whole pipeline to data
results = clf.predict(test)

In [32]:
result = pd.DataFrame(data=results)
result.head()

,0
0,-0.188574
1,-0.519521
2,0.827562
3,-0.196114
4,1.960363


In [33]:
# Applying inverse power transform to get back original form of unit price
a_inverse_transformed = power.inverse_transform(result)
a_inverse_transformed

array([[1.65      ],
       [1.25      ],
       [3.88417208],
       ...,
       [1.25      ],
       [6.31501274],
       [4.15      ]])

In [34]:
result = pd.DataFrame(data=a_inverse_transformed).round(2)
result.head()

,0
0,1.65
1,1.25
2,3.88
3,1.64
4,12.75


In [35]:
result.to_excel("UnitPricePrediction.xlsx")
